# Player Rating

## 1. Process Data

In [3]:
import os
from pathlib import Path
import pandas as pd
import datetime
from datetime import datetime

In [4]:

# NOTE: This wont work for jupyter so we are using sth else for now
# # Get the absolute path of the current script (jupyter dir)
# BASE_DIR = Path(__file__).resolve().parent

# # Build the path to the data directory
# DATA_DIR = BASE_DIR / 'data'
BASE_DIR = Path.cwd()

# Build the path to the data directory
DATA_DIR = BASE_DIR.parents[0] / 'transfer_data' 

# # Example of using the path to the data directory
# csv_file = DATA_DIR / 'your_file.csv'
# print(csv_file)

In [5]:
# import all files in Data folder and read into dataframes
dataframes=[]


# Below is just to surpress warnings...
competitions_df = pd.DataFrame
appearances_df = pd.DataFrame
player_valuations_df = pd.DataFrame
game_events_df = pd.DataFrame
players_df= pd.DataFrame
games_df = pd.DataFrame
club_games_df = pd.DataFrame
clubs_df = pd.DataFrame

# Actual reading csv flies
for dirpath, dirname, filenames in os.walk(DATA_DIR):
    for filename in filenames:
        file=filename.split('.')
        file=((file[0]+"_df"))
        if file !="_df":
            filepath=os.path.join(dirpath,filename)
            df=pd.read_csv(filepath,sep=",",encoding = "UTF-8")
            exec(f'{file} = df.copy()')
            print(file, df.shape)
            dataframes.append(df)
print('Data imported')



game_lineups_df (2191911, 10)
competitions_df (44, 11)
appearances_df (1610495, 13)
player_valuations_df (486009, 5)
game_events_df (761513, 10)
transfers_df (77585, 10)
players_df (32392, 23)
games_df (69635, 23)
club_games_df (139270, 11)
club_elos_df (6386, 9)
clubs_df (439, 17)
Data imported


# Process data

## Players_df

In [6]:
players_df.shape

(32392, 23)

In [7]:
# Calculate the age of each player (object -> datetime)
players_df['date_of_birth'] = pd.to_datetime(players_df['date_of_birth']) 
# drop players with no date of birth
players_df.dropna(subset = ['date_of_birth'], inplace=True)
players_df.shape

(32345, 23)

In [8]:
# Get age column of players
now = datetime.now()
players_df['age'] = (now - players_df['date_of_birth']).apply(lambda x: x.days / 365.25)
players_df['age'] = players_df['age'].round().astype(int)
# players_df['age'] = players_df['age'].round().astype(int) 


In [9]:
elo_df = players_df.copy()


# Initialisation
## 1. Getting club elo
Using clubelo.com's API, get club elo

In [10]:
import io
import requests

# Get a date and data
response = requests.get(f"http://api.clubelo.com/{datetime.today().strftime('%Y-%m-%d')}")
club_elo_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
club_elo_df.drop('Rank', axis=1)

,Club,Country,Level,Elo,From,To
0,Man City,ENG,1,2060.798584,2024-09-20,2024-09-22
1,Real Madrid,ESP,1,1995.374390,2024-09-22,2024-09-24
2,Inter,ITA,1,1963.372681,2024-09-20,2024-09-22
3,Arsenal,ENG,1,1959.767212,2024-09-20,2024-09-22
4,Leverkusen,GER,1,1939.445068,2024-09-20,2024-09-22
...,...,...,...,...,...,...
634,FCB Magpies,GIB,0,896.816284,2024-08-30,2024-10-06
635,Atletic Club Escaldes,AND,0,880.255310,2024-08-30,2024-10-06
636,SS Virtus,SMR,0,708.203552,2024-08-02,2024-10-06
637,Tre Penne,SMR,0,695.522400,2024-08-02,2024-10-06


## 2. initialise player
### 2-1. Find oldest / newest match
- And decide where should we initialise

In [11]:
games_df.shape
games_df.dtypes

game_id                     int64
competition_id             object
season                      int64
round                      object
date                       object
home_club_id                int64
away_club_id                int64
home_club_goals             int64
away_club_goals             int64
home_club_position        float64
away_club_position        float64
home_club_manager_name     object
away_club_manager_name     object
stadium                    object
attendance                float64
referee                    object
url                        object
home_club_formation        object
away_club_formation        object
home_club_name             object
away_club_name             object
aggregate                  object
competition_type           object
dtype: object

In [12]:
# Change data type to datetime format
games_df['date'] = games_df['date'].str.strip()
games_df['date'] = pd.to_datetime(games_df['date'], errors='coerce', format='%Y-%m-%d')
games_df.dropna(subset=['date'], inplace=True)

# Sort by date
games_df = games_df.sort_values(by=['date'])
games_df.head()

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
14610,2234412,ELQ,2012,First Round 1st leg,2012-07-03,24832,10694,0,6,NaN,...,Gundadalur,650.0,Andris Treimanis,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:6,international_cup
14605,2231979,CLQ,2012,First Round 1st leg,2012-07-03,3471,10532,0,0,NaN,...,Windsor Park,1341.0,Wim Smet,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
43075,2231978,CLQ,2012,First Round 1st leg,2012-07-03,853,10747,7,0,NaN,...,Stade Jos Nosbaum,1070.0,Alan Mario Sant,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,7:0,international_cup
10890,2235764,ELQ,2012,First Round 1st leg,2012-07-03,1167,5594,0,0,NaN,...,Milano Arena,2000.0,Eitan Shmuelevic,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,0:0,international_cup
46883,2231982,CLQ,2012,First Round 1st leg,2012-07-03,6335,28958,8,0,NaN,...,Tony Bezzina Stadium,1136.0,Johnny Casanova,https://www.transfermarkt.co.uk/spielbericht/i...,NaN,NaN,NaN,NaN,8:0,international_cup


In [13]:
# games_df.sort_values(by=['date'])
oldest_date = games_df['date'].min()
newest_date = games_df['date'].max()

In [14]:
oldest_date.strftime('%Y-%m-%d')

'2012-07-03'

Add elo column to players df and initialise it to club elo of that time

In [15]:
# Get a date and data
response = requests.get(f"http://api.clubelo.com/{oldest_date.strftime('%Y-%m-%d')}")
clubs_elo_df = pd.read_csv(io.StringIO(response.content.decode('utf-8')))
clubs_elo_df.drop('Rank', axis=1)

,Club,Country,Level,Elo,From,To
0,Real Madrid,ESP,1,2089.266357,2012-05-14,2012-08-02
1,Barcelona,ESP,1,2072.035156,2012-05-13,2012-08-02
2,Bayern,GER,1,1928.243896,2012-05-20,2012-08-02
3,Man City,ENG,1,1917.816162,2012-05-20,2012-08-02
4,Dortmund,GER,1,1914.006226,2012-05-20,2012-08-02
...,...,...,...,...,...,...
632,Lusitanos,AND,0,769.535339,2011-07-22,2012-07-03
633,Escale,AND,0,768.268860,2011-07-22,2012-07-03
634,Libertas,SMR,0,716.972961,2012-07-01,2012-07-03
635,La Fiorita,SMR,0,716.972961,2012-07-01,2012-07-03


In [16]:
players_df.columns

Index(['player_id', 'first_name', 'last_name', 'name', 'last_season',
       'current_club_id', 'player_code', 'country_of_birth', 'city_of_birth',
       'country_of_citizenship', 'date_of_birth', 'sub_position', 'position',
       'foot', 'height_in_cm', 'contract_expiration_date', 'agent_name',
       'image_url', 'url', 'current_club_domestic_competition_id',
       'current_club_name', 'market_value_in_eur',
       'highest_market_value_in_eur', 'age'],
      dtype='object')

In [17]:
transfers_df

,player_id,transfer_date,transfer_season,from_club_id,to_club_id,from_club_name,to_club_name,transfer_fee,market_value_in_eur,player_name
0,195778,2026-06-30,25/26,79,27,VfB Stuttgart,Bayern Munich,0.0,12000000.0,Alexander Nübel
1,569033,2026-06-30,25/26,39,27,1.FSV Mainz 05,Bayern Munich,0.0,4000000.0,Armindo Sieb
2,626913,2026-06-30,25/26,398,380,Lazio,Salernitana,0.0,10000000.0,Boulaye Dia
3,1047109,2026-06-30,25/26,2672,265,APO Levadiakos,Panathinaikos,0.0,100000.0,Georgios Katris
4,360791,2025-12-31,25/26,6418,1030,Panetolikos,Argentinos Jrs.,0.0,1200000.0,Lucas Chaves
...,...,...,...,...,...,...,...,...,...,...
77580,35207,1995-07-01,95/96,42281,11896,PTSV Dort. Yth.,Dortmund Yth.,0.0,NaN,Marco Reus
77581,22141,1994-07-01,94/95,59178,29463,Castellammare,Sorrento Youth,NaN,NaN,Antonio Mirante
77582,33829,1994-07-01,94/95,58405,53957,FC Penafiel You,Sporting Yth.,NaN,NaN,José Fonte
77583,40680,1994-07-01,94/95,47320,47318,Lichtenw. Yth.,Schornbach Yth.,0.0,NaN,Sven Ulreich


In [18]:
players_df.columns

Index(['player_id', 'first_name', 'last_name', 'name', 'last_season',
       'current_club_id', 'player_code', 'country_of_birth', 'city_of_birth',
       'country_of_citizenship', 'date_of_birth', 'sub_position', 'position',
       'foot', 'height_in_cm', 'contract_expiration_date', 'agent_name',
       'image_url', 'url', 'current_club_domestic_competition_id',
       'current_club_name', 'market_value_in_eur',
       'highest_market_value_in_eur', 'age'],
      dtype='object')

In [19]:
df = players_df.copy()

---

### Player Initialisation
Find a **debut** match of players

In [20]:
club_elos_df = pd.read_csv(DATA_DIR / 'club_elos.csv')

#### Test
Lets test with a single case

In [21]:
filtered_df = players_df[players_df['name'].str.contains('Ronaldo', case=False, na=False) & players_df['name'].str.contains('Cristiano', case=False, na=False)].copy()
# filtered_df = players_df[players_df['name'].str.contains('Ronaldo', case=False, na=False)]
filtered_df

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur,age
758,8198,NaN,Cristiano Ronaldo,Cristiano Ronaldo,2022,985,cristiano-ronaldo,Portugal,Funchal,Portugal,...,187.0,2025-06-30 00:00:00,Gestifute,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/cristiano-rona...,GB1,Manchester United Football Club,15000000.0,120000000.0,40


In [22]:
player_id = filtered_df.iloc[0].player_id
df = appearances_df[appearances_df['player_id'] == player_id].copy()

In [23]:
# Clean date type
# Change data type to datetime format
df['date'] = df['date'].str.strip()
df['date'] = pd.to_datetime(df['date'], errors='coerce', format='%Y-%m-%d')
df.dropna(subset=['date'], inplace=True)

# Sort by date
sorted_df = df.sort_values(by=['date'])
sorted_df.head()

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
8238,2244382_8198,2244382,8198,418,985,2012-08-19,Cristiano Ronaldo,ES1,0,0,0,0,90
8948,2244388_8198,2244388,8198,418,985,2012-08-22,Cristiano Ronaldo,SUC,0,0,1,0,90
11986,2242834_8198,2242834,8198,418,985,2012-08-26,Cristiano Ronaldo,ES1,0,0,0,0,90
12902,2244389_8198,2244389,8198,418,985,2012-08-29,Cristiano Ronaldo,SUC,0,0,1,0,90
16015,2242882_8198,2242882,8198,418,985,2012-09-02,Cristiano Ronaldo,ES1,0,0,2,0,63


NOTE: We know ronaldo didn't make his debut at 2012 but this is how far our data covers so...
Now get elo of ronaldo's team at that year

In [30]:
# Assume you want to select the first row from appearances_df
selected_row_date = pd.to_datetime(sorted_df.iloc[0]['date'])

# Get team name
club_id = sorted_df.iloc[0]['player_club_id']

# Get the team name from teams_df based on club_id
club_name_row = clubs_df[clubs_df['club_id'] == club_id]

if not club_name_row.empty:
    team_name = club_name_row['name'].values[0]  # Get the team name
    print(f"Selected Team Name: {team_name}")

    # Convert the 'date' column in club_elos to datetime format
    club_elos_df['date'] = pd.to_datetime(club_elos_df['date'])

    # Filter club_elos_df for the selected team name
    filtered_club_elos = club_elos_df[club_elos_df[''].str]
    
    players_df[players_df['name'].str.contains('Ronaldo', case=False, na=False)]

    # Find the closest date in the filtered DataFrame
    if not filtered_club_elos.empty:
        closest_row = filtered_club_elos.iloc[(filtered_club_elos['date'] - selected_row_date).abs().argmin()]
        print(closest_row)
    else:
        print("No matching records found in club_elos_df for the selected team.")
else:
    print("No team found with the specified team_id.")
# # Convert the 'date' column in club_elos to datetime format
# club_elos_df['Updated'] = pd.to_datetime(club_elos_df['Updated'])

# # Find the closest Updated
# closest_row = club_elos_df[(club_elos_df['Updated'] - selected_row_date).abs().argmin()].copy()


,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
289,418,real-madrid,Real Madrid Club de Fútbol,ES1,NaN,22,26.6,17,77.3,18,Santiago Bernabéu,81044,€-33.00m,NaN,2024,../data/raw/transfermarkt-scraper/2024/clubs.j...,https://www.transfermarkt.co.uk/real-madrid/st...


In [60]:
appearances_df.query('player_id')
# # Calculate the contract remaining of each player
# players_df['contract_expiration_date'] = pd.to_datetime(players_df['contract_expiration_date'])
# # drop players with no date of birth
# players_df = players_df[players_df['contract_expiration_date'].isnull() == False]
# now = datetime.now()
# players_df['term_days_remaining'] = (players_df['contract_expiration_date']- now).apply(lambda x: x.days) 



/var/folders/nh/kbd4vyw93l9_cfg59h74rpwh0000gn/T/ipykernel_4550/1132561174.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  players_df['term_days_remaining'] = (players_df['contract_expiration_date']- now).apply(lambda x: x.days)


ValueError: unconverted data remains when parsing with format "%Y-%m-%d": " 00:00:00", at position 0. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [59]:
players_df

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,height_in_cm,market_value_in_eur,highest_market_value_in_eur,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,age
0,597,Aleksandr,Hleb,Aleksandr Hleb,2016,2696,aleksandr-hleb,UdSSR,Minsk,Belarus,...,NaN,NaN,19000000.0,NaN,fair-sport GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aleksandr-hleb...,RU1,Krylya Sovetov Samara,43
1,1428,Mike,Hanke,Mike Hanke,2013,60,mike-hanke,Germany,Hamm,Germany,...,185.0,NaN,5000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/mike-hanke/pro...,L1,SC Freiburg,41
2,1560,Youssef,El Akchaoui,Youssef El Akchaoui,2012,306,youssef-el-akchaoui,Netherlands,Dordrecht,Morocco,...,NaN,NaN,1750000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/youssef-el-akc...,NL1,SC Heerenveen,44
4,1726,Heiko,Westermann,Heiko Westermann,2016,610,heiko-westermann,Germany,Alzenau,Germany,...,190.0,NaN,9000000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/heiko-westerma...,NL1,Ajax Amsterdam,41
5,2306,Markus,Miller,Markus Miller,2014,42,markus-miller,Germany,Lindenberg im Allgäu,Germany,...,189.0,NaN,3500000.0,NaN,ARP Sportmarketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/markus-miller/...,L1,NaN,42
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28454,199337,Rifat,Erdem,Rifat Erdem,2013,2293,rifat-erdem,Turkey,Konya,Turkey,...,NaN,NaN,100000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/rifat-erdem/pr...,TR1,Konyaspor,35
28455,209569,Mücahit,Atalay,Mücahit Atalay,2020,2293,mucahit-atalay,Turkey,Trabzon,Turkey,...,186.0,NaN,250000.0,2023-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/mucahit-atalay...,TR1,Konyaspor,33
28456,221118,Patrick Friday,Eze,Patrick Friday Eze,2017,2293,patrick-friday-eze,Nigeria,Kaduna,Nigeria,...,187.0,200000.0,775000.0,2023-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/patrick-friday...,TR1,Konyaspor,32
28457,273053,Burak,Saban,Burak Saban,2018,2293,burak-saban,Netherlands,Hoorn,Netherlands,...,177.0,NaN,200000.0,NaN,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/burak-saban/pr...,TR1,Konyaspor,30


In [ ]:
# drop players with no date of birth
# players_df = players_df[players_df['date_of_birth'].isnull() == False]
# now = datetime.now()
# players_df['age'] = (now - players_df['date_of_birth']).apply(lambda x: x.days) / 365.25
# players_df['age'] = players_df['age'].round().astype(int) 

# # Calculate the contract remaining of each player
# players_df['contract_expiration_date'] = pd.to_datetime(players_df['contract_expiration_date'])
# # drop players with no date of birth
# players_df = players_df[players_df['contract_expiration_date'].isnull() == False]
# now = datetime.now()
# players_df['term_days_remaining'] = (players_df['contract_expiration_date']- now).apply(lambda x: x.days) 

# # add year to player valuations
# player_valuations_df['datetime']=pd.to_datetime(player_valuations_df['datetime'], format="%Y-%m-%d")
# player_valuations_df['year']=player_valuations_df['datetime'].dt.year

# # add year to player appearances
# appearances_df['datetime']=pd.to_datetime(appearances_df['date'], format="%Y-%m-%d")
# appearances_df['year']=appearances_df['datetime'].dt.year

# #add position to player valuations
# position_df=players_df.copy()
# position_df=position_df.drop(['name', 'current_club_id', 'current_club_name', 'country_of_citizenship', 'country_of_birth', 'city_of_birth', 'date_of_birth', 'foot', 'height_in_cm', 'market_value_in_eur', 'highest_market_value_in_eur', 'agent_name', 'contract_expiration_date', 'current_club_domestic_competition_id', 'first_name', 'last_name', 'player_code', 'image_url', 'last_season', 'url', 'age'], axis=1)
# player_valuations_df=player_valuations_df.merge(position_df, left_on='player_id', right_on='player_id')

# #add position to appearances
# value_df=players_df.copy()
# value_df=value_df.drop(['name', 'current_club_id', 'current_club_name', 'country_of_citizenship', 'country_of_birth', 'city_of_birth', 'date_of_birth', 'foot', 'height_in_cm', 'sub_position', 'highest_market_value_in_eur', 'agent_name', 'contract_expiration_date', 'current_club_domestic_competition_id', 'first_name', 'last_name', 'player_code', 'image_url', 'last_season', 'url', 'age'], axis=1)
# appearances_df1=value_df.merge(appearances_df, left_on='player_id', right_on='player_id')

# print("Data processing complete")